In [1]:
# code gen + values list

#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
#from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


In [2]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage
llm = ChatOllama(model="qwen3:1.7b")  # vagy bármely más elérhető Ollama modell

In [3]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

#llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
set_llm_cache(SQLiteCache(database_path=".langchain.ollama25.db"))

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [5]:
def get_question(qid):
    for i, item in devdf.iterrows():
        for q in item['questions']:        
            if q['uid'] == qid:
                #table = item['table']['table']
                return (item['table'], q)
    return (None, None)

In [12]:
import re
def extract_code_blocks(text):
    return "\n\n".join(re.findall(r'```(?:\w*\n)?(.*?)```', text, re.DOTALL))

def gen_code3(llm, messages, question, value_list): 

    messages[0] = ( messages[0][0],  "/no_think " +  messages[0][1] ) 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"value_list": value_list, "question":question})
    print("R: ", response + "|")
    
    if "```python" in response:        
        code =  extract_code_blocks(response)

    
    elif "</think>" in response:
        idx = response.rfind("</think>") + len("</think>") 
        code =  response[idx:].strip()
    else:
        code =  response.strip()
        
    #code =  response.replace('```python','').replace('```','')
    print("code: ", code + "|")
    
    return (prompt.format(value_list = value_list, question = question), code)

def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if "run()" in code:
                exec(code + "\nr = run()\n", globals(), loc)
            elif "def run(" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)                
            else : 
                exec("r = " + code + "\n", globals(), loc)                            
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')

def gen_code_table(llm, messages, question, table): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"table": table, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return (prompt.format(table = table, question = question), code)

def exec_code_table(code, table):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({table})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')

In [13]:
def get_answer(llm, messages, table, q):
    values = table_convert.convert_multitable(table)    
    p, code = gen_code3(llm, messages, q, values)    
    #print(code)
    (v, s) = exec_code(code, values)
    return  (v, s, code)

In [14]:
import prompt_versions
items = [i for idx, i in devdf.iterrows()]
item = items[2]
table = item['table']['table']
q = item.questions[3]
question = q['question']
#print(table, question)
r = get_answer(llm, prompt_versions.prompt_versions['V11'], table, question)
print(r[0], r[1], r[2])


R:   

def run(value_list):
    # Find the discount rates for domestic and international in 2019
    domestic_rate = None
    international_rate = None
    for item in value_list:
        if item['header3'] == 'Domestic' and item['header1'] == '2019':
            domestic_rate = item['number_value']
        elif item['header3'] == 'International' and item['header1'] == '2019':
            international_rate = item['number_value']
    # Calculate the difference
    difference = domestic_rate - international_rate
    # Determine the scale
    if difference == 0:
        scale = ''
    else:
        # Check if the difference is in thousands, millions, billions
        scale = ''
        if abs(difference) >= 1000000000:
            scale = 'billion'
        elif abs(difference) >= 1000000:
            scale = 'million'
        elif abs(difference) >= 1000:
            scale = 'thousand'
        else:
            scale = ''
    return (difference, scale)

Wait, but the question asks for th

In [ ]:
from tqdm.notebook import tqdm as log_progress
import prompt_versions

devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
cnt = 0
res = []
for i, item in log_progress(devdf.iterrows()):
    try:
        #if cnt == 36:
        #    break;
        #print(item['table']['uid'])
        table = item['table']['table']
        value_list = table_convert.convert_multitable(table)    
        text = '\n'.join( [p['text'] for p in item['paragraphs']])
        for q in item['questions']:        
            if q['answer_type'] == 'arithmetic' and 'table' == q['answer_from']:   
                cnt = cnt + 1
                print(q['uid'])
                _table, _q =  get_question(q['uid'])
                
                table = _table['table']
                
                #[print(r) for r in table]
                
                q = _q['question']
                print(q, end='')
                
                
                r = []
                #for i in [0,1,2]:        
                #   r.append(get_answer(llm, messages, table, q))
                
                #if r[0][0] == r[1][0]:
                #    (v, s) = r[0]
                #elif r[1][0] == r[2][0]:
                #    (v, s) = r[1]
                #elif r[0][0] == r[2][0]:
                #    (v, s) = r[0]
                #else:
                #    (v, s) = (None, None)
                    
                #(v, s, code) = get_answer(llm, messages, table, q)    
                (pred_value, pred_scale, code) = get_answer(llm, prompt_versions.prompt_versions['V11'], table, q)    
                
                if 'in millions' in text.lower() and pred_scale == '':
                    pred_scale = 'million'
                    
                if  utils.eval_predicted_value(pred_value, _q['answer']):
                    print("\033[92m Success: " + str(pred_value)+'\033[0m')
                else:    
                    print("\033[91m failure: " + str(pred_value), 'good answer: ', _q['answer'],'\033[0m' )
                
                if isinstance(pred_value, tuple) and len(pred_value) == 2:
                    print('$$$$')
                    (pred_value, pred_scale) = pred_value
                if pred_scale == "%" or pred_scale == "percentage"  :
                    pred_scale = 'percent'
                if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                    print('Invalid ', pred_scale)
                    pred_scale = ""    
                #if item['table']['uid'] in res_scale and pred_scale != 'percent':
                #    pred_scale = res_scale[item['table']['uid']]
                err=""
                if isinstance(pred_value, str):
                    #print("string")
                    if  pred_value.startswith('[Error]'):
                        err = pred_value
                        (pred_value, pred_scale) = ("", "")
                                                                              
                res.append(({"answer_type":_q["answer_type"], "answer": _q["answer"], 'scale': _q["scale"]}, pred_value, pred_scale, _q, code, table,value_list, err))
    except Exception as e:
        s = '[Outer Exception]'+ str(e)
        print(s)
        res.append(({"answer_type":_q["answer_type"], "answer": _q["answer"], 'scale': _q["scale"]}, "", "", _q, "", table,"", s))

In [39]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

0.17615176151761516 0.17615176151761516 0.3035230352303523


In [40]:
import re
res2 = []
for ans, pred, pred_scale, q,code, table,value_list,err in res:
    good = utils.eval_predicted_value(pred, ans['answer'])
    regex = re.compile("\\(\\d+,?\\d+", re.VERBOSE)    
    match = regex.search(str(table))
    
    match = match != None
    if pred == None:
        pred = 0
    res2.append( {'qid': q['uid'], 'question' : q['question'], 'derivation': q['derivation'], 'pred' : pred, 'pred_scale': pred_scale,  'answer': ans['answer'],  'scale':  ans['scale'],  'abs': 'abs(' in code, 'inv' : ans['answer'] == -1*pred,  'good': good, 'parenth': match})
res2 = pd.DataFrame(res2)
print(len(res2.query('good == True'))/len(res2))

0.2926829268292683


In [171]:
metrics._details

[{'answer_type': 'arithmetic',
  'answer': -94,
  'scale': 'million',
  'pred': [-94.0],
  'pred_scale': 'million',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'arithmetic',
  'answer': -12.14,
  'scale': 'percent',
  'pred': [-12.14],
  'pred_scale': 'percent',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'arithmetic',
  'answer': 2.1,
  'scale': 'percent',
  'pred': [2.1],
  'pred_scale': 'percent',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'arithmetic',
  'answer': 6.67,
  'scale': 'percent',
  'pred': [6.67],
  'pred_scale': 'percent',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'arithmetic',
  'answer': -8.11,
  'scale': 'percent',
  'pred': [-8.11],
  'pred_scale': 'percent',
  'em': 1.0,

In [172]:
metrics._details[0]['pred'][0] == -94

True

In [207]:
scale_err =  [item for item in metrics._details if item['em'] != 1.0 and item['pred_scale'] != item['scale']]
only_scale_err =  [item for item in metrics._details if item['em'] != 1.0 and isinstance(item['pred'], list)  and  item['pred'][0] == item['answer']]
err =  [item for item in metrics._details if item['em'] != 1.0]
len (metrics._details), len(err), len(scale_err), len(only_scale_err)

(166, 56, 31, 20)

In [198]:
res = {}
for item in metrics._details:
  

    pred = item['pred']
    pred_scale = item['pred_scale']

    key = pred_scale +'|' + item['scale']
    if key not in res:
        res [key] = 1
    res[key]=res[key] +1


    if  item['em'] == 1.0:
        None
        #print("\033[92m Success: " + str(pred)+'\033[0m')
    else:           
 
        
        if isinstance(item['pred'], list) and  pred[0] == item['answer']:
           
            print("\033[93m", '|', " failure: ", '| ',str(pred[0]),  '| ', str(pred_scale), '| ', 'good answer: ', '| ', item['answer'], '| ', item['scale'], '\033[0m' )
        else:    
            None
            #print("\033[91m failure: " + str(pred),  str(pred_scale), 'good answer: ',item['answer'],item['scale'], '\033[0m' )

 |  failure:  |  62740.0 |   |  good answer:  |  62740 |  thousand 
 |  failure:  |  -19411.0 |   |  good answer:  |  -19411 |  thousand 
 |  failure:  |  12085176.0 |  thousand |  good answer:  |  12085176 |   
 |  failure:  |  28747.0 |  thousand |  good answer:  |  28747 |   
 |  failure:  |  113000.0 |  thousand |  good answer:  |  113000 |   
 |  failure:  |  111.4 |  million |  good answer:  |  111.4 |   
 |  failure:  |  107.15 |  million |  good answer:  |  107.15 |   
 |  failure:  |  -27603.0 |  thousand |  good answer:  |  -27603 |   
 |  failure:  |  3.0 |  million |  good answer:  |  3 |   
 |  failure:  |  -10029.67 |   |  good answer:  |  -10029.67 |  thousand 
 |  failure:  |  39.33 |  million |  good answer:  |  39.33 |   
 |  failure:  |  0.08 |  thousand |  good answer:  |  0.08 |  percent 
 |  failure:  |  16767.0 |   |  good answer:  |  16767 |  thousand 
 |  failure:  |  0.1 |   |  good answer:  |  0.1 |  million 
 |  failure:  |  54633874 |  thousand |  good answ

In [199]:
res

{'million|million': 121,
 'percent|percent': 177,
 '|': 65,
 'thousand|thousand': 82,
 '|thousand': 11,
 '|percent': 14,
 'percent|': 3,
 '|million': 5,
 'thousand|': 11,
 'million|percent': 5,
 'million|': 9,
 'thousand|percent': 2,
 'billion|billion': 4,
 'million|thousand': 2}

In [191]:
res


{'|thousand': 5,
 'thousand|': 10,
 'million|': 4,
 'thousand|percent': 2,
 '|million': 7,
 'million|thousand': 2,
 'million|percent': 3}

In [67]:
metrics._details

[{'answer_type': 'arithmetic',
  'answer': -94,
  'scale': 'million',
  'pred': [-94.0],
  'pred_scale': 'million',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'arithmetic',
  'answer': -12.14,
  'scale': 'percent',
  'pred': [-12.14],
  'pred_scale': 'percent',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'arithmetic',
  'answer': 2.1,
  'scale': 'percent',
  'pred': [2.1],
  'pred_scale': 'percent',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'arithmetic',
  'answer': 6.67,
  'scale': 'percent',
  'pred': [6.67],
  'pred_scale': 'percent',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'arithmetic',
  'answer': -8.11,
  'scale': 'percent',
  'pred': [-8.11],
  'pred_scale': 'percent',
  'em': 1.0,